## ITVERSITY CCA-175 Practice

## For LOCAL MACHINE SETUP

In [4]:
import findspark as fs
import os
fs.init()
fs.find()

data_path=os.path.dirname("D://Bigdata Tutorials//data//retail_db//")
data_path_json=os.path.dirname("D://Bigdata Tutorials//data//retail_db_json//")

from pyspark.sql import SparkSession,SQLContext,HiveContext
spark=SparkSession.builder.appName('ITVERSITY').master('local').getOrCreate()
sc=spark.sparkContext
sqlcontext=SQLContext(sc)
sc

<SparkContext master=local appName=ITVERSITY>

## For VAGRANT SETUP

In [1]:
!start-dfs.sh
!start-yarn.sh

from pyspark.sql import SparkSession,SQLContext,HiveContext
spark=SparkSession.builder.appName('ITVERSITY').master('yarn').getOrCreate()
sc=spark.sparkContext
sqlcontext=SQLContext(sc)
sc

<SparkContext master=yarn appName=ITVERSITY>

In [4]:
orderItems=sc.textFile(os.path.join(data_path,"order_items"))

In [ ]:
orderItems=sc.textFile("/public/retail_db/order_items")


In [5]:
type(orderItems)

pyspark.rdd.RDD

In [7]:
# It contains OrderID
int(orderItems.first().split(",")[1])

1

In [8]:
# It contains Revenue per order Item
float(orderItems.first().split(",")[4])

299.98

In [52]:
orderItemsMap=orderItems.map(lambda x:(int(x.split(",")[1]),float(x.split(",")[4])))

In [53]:
orderItemsMap.take(5)

[(1, 299.98), (2, 199.99), (2, 250.0), (2, 129.99), (4, 49.98)]

In [54]:
# Now we can hold 1st element as KEY and reduce the Values using add.

In [61]:
from operator import add
revenuePerOrder=orderItemsMap.reduceByKey(add)

In [62]:
for i in revenuePerOrder.take(10): print(i)

(1, 299.98)
(2, 579.98)
(4, 699.85)
(5, 1129.8600000000001)
(7, 579.9200000000001)
(8, 729.8399999999999)
(9, 599.96)
(10, 651.9200000000001)
(11, 919.79)
(12, 1299.8700000000001)


### Transformations follow LazyEvaluation.

#### LazyEvaluation simply uses a DAG(Directed Acyclic Graph) to store all the information related to the Transformations being made.

#### As soon as an Action is 'run', spark executes the DAG first and then the action.

Let's find DAG of transformations:


In [ ]:
orderItems.toDebugString()

In [ ]:
orderItemsMap.toDebugString()

In [ ]:
revenuePerOrder.toDebugString()

### Now we can open spark job UI and look into DAG visualization.


### NOTE: DO NOT USE collect() to preview data in REAL LIFE ENVIRONMENT!!!

### Another way tp create an RDD is to open a file using open and making a LIST out of it. (a collection)

In [ ]:
productsRaw=open(data_path+"//products").read().splitlines()

In [ ]:
type(productsRaw)

### Now, we can create an RDD using Parallelize from the collection.

In [ ]:
productsRaw=sc.parallelize(productsRaw)

In [ ]:
type(productsRaw)

In [ ]:
print(productsRaw.first())
type(productsRaw.first())

## DATA FRAME

#### --provided by sqlContext

#### Now we can try loading multiple file formats.

In [ ]:
sqlcontext.read?

##same as spaek.read?

In [12]:
df = spark.read.json(data_path_json+"//order_items")

In [13]:
df.show(5)

+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_product_price|order_item_quantity|order_item_subtotal|
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|            1|                  1|                  957|                  299.98|                  1|             299.98|
|            2|                  2|                 1073|                  199.99|                  1|             199.99|
|            3|                  2|                  502|                    50.0|                  5|              250.0|
|            4|                  2|                  403|                  129.99|                  1|             129.99|
|            5|                  4|                  897|                   24.99|                  2|              49.98|
+-------------+-

In [17]:
df1=spark.read.format("json").load(data_path_json+"//order_items")

In [18]:
df1.show(5)

+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_product_price|order_item_quantity|order_item_subtotal|
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|            1|                  1|                  957|                  299.98|                  1|             299.98|
|            2|                  2|                 1073|                  199.99|                  1|             199.99|
|            3|                  2|                  502|                    50.0|                  5|              250.0|
|            4|                  2|                  403|                  129.99|                  1|             129.99|
|            5|                  4|                  897|                   24.99|                  2|              49.98|
+-------------+-

## Orders and OrderItems Example

In [39]:
orders=sc.textFile(os.path.join(data_path,"orders"))
orders.first()

u'1,2013-07-25 00:00:00.0,11599,CLOSED'

In [86]:
orders.take(5)

[u'1,2013-07-25 00:00:00.0,11599,CLOSED',
 u'2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 u'3,2013-07-25 00:00:00.0,12111,COMPLETE',
 u'4,2013-07-25 00:00:00.0,8827,CLOSED',
 u'5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [87]:
orders.count()

68883

### Extract OrderStatus: (YYYYMMDD,count)

In [109]:
ordersMap=orders.map(lambda x: (x.split(",")[1].split(" ")[0].replace("-",""),x.split(",")[3]))
ordersMap.first()

(u'20130725', u'CLOSED')

In [101]:
# To get data of COMPLETE ORDERS OR CLOSED orders
ordersComplete=orders.filter(lambda x: x.split(",")[3] == 'COMPLETE' or x.split(",")[3] == 'CLOSED')
ordersComplete.take(5)

[u'1,2013-07-25 00:00:00.0,11599,CLOSED',
 u'3,2013-07-25 00:00:00.0,12111,COMPLETE',
 u'4,2013-07-25 00:00:00.0,8827,CLOSED',
 u'5,2013-07-25 00:00:00.0,11318,COMPLETE',
 u'6,2013-07-25 00:00:00.0,7130,COMPLETE']

In [103]:
#To get CLOSED/COMPLETE orders in 2014-01
ordersComplete=orders.filter(lambda x: (x.split(",")[1][:7]=='2014-01') and (x.split(",")[3] in ['COMPLETE','CLOSED'] ))
ordersComplete.take(5)

[u'25882,2014-01-01 00:00:00.0,4598,COMPLETE',
 u'25888,2014-01-01 00:00:00.0,6735,COMPLETE',
 u'25889,2014-01-01 00:00:00.0,10045,COMPLETE',
 u'25891,2014-01-01 00:00:00.0,3037,CLOSED',
 u'25895,2014-01-01 00:00:00.0,1044,COMPLETE']